In [1]:
import numpy as np
from difflib import SequenceMatcher
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from janome.tokenizer import Tokenizer
from datasketch import MinHash, MinHashLSH
import simhash
import spacy
import re

#######################################
# 基本的なカバー率計算の各手法
#######################################

def simple_coverage(list_a, list_b, threshold=0.5):
    """文字列類似度ベースのカバー率計算"""
    a_covered = sum(1 for a in list_a if any(SequenceMatcher(None, a, b).ratio() >= threshold for b in list_b))
    b_covered = sum(1 for b in list_b if any(SequenceMatcher(None, a, b).ratio() >= threshold for a in list_a))
    
    return a_covered / len(list_a), b_covered / len(list_b)

def ngram_coverage(list_a, list_b, n=2, threshold=0.3):
    """文字n-gramのJaccard係数によるカバー率計算"""
    def get_ngrams(text, n):
        return set(text[i:i+n] for i in range(len(text) - n + 1))
    
    ngrams_a = [get_ngrams(text, n) for text in list_a]
    ngrams_b = [get_ngrams(text, n) for text in list_b]
    
    a_covered = sum(1 for a_set in ngrams_a if 
                   any(len(a_set & b_set) / len(a_set | b_set) >= threshold for b_set in ngrams_b))
    b_covered = sum(1 for b_set in ngrams_b if 
                   any(len(a_set & b_set) / len(a_set | b_set) >= threshold for a_set in ngrams_a))
    
    return a_covered / len(list_a), b_covered / len(list_b)

def janome_coverage(list_a, list_b, threshold=0.4):
    """Janomeによる形態素解析を使用したカバー率計算"""
    tokenizer = Tokenizer()
    
    # 内容語のみを抽出
    def get_content_words(text):
        content_pos = ('名詞', '動詞', '形容詞', '副詞')
        return {t.base_form for t in tokenizer.tokenize(text) 
                if t.part_of_speech.split(',')[0] in content_pos}
    
    tokens_a = [get_content_words(text) for text in list_a]
    tokens_b = [get_content_words(text) for text in list_b]
    
    a_covered = sum(1 for a_tokens in tokens_a if 
                   any(len(a_tokens & b_tokens) / len(a_tokens | b_tokens) >= threshold 
                      for b_tokens in tokens_b if a_tokens and b_tokens))
    b_covered = sum(1 for b_tokens in tokens_b if 
                   any(len(a_tokens & b_tokens) / len(a_tokens | b_tokens) >= threshold 
                      for a_tokens in tokens_a if a_tokens and b_tokens))
    
    return a_covered / len(list_a), b_covered / len(list_b)

def tfidf_coverage(list_a, list_b, threshold=0.5):
    """TF-IDFベクトル化と余弦類似度を用いたカバー率計算"""
    vectorizer = TfidfVectorizer(analyzer='char', ngram_range=(2, 3))
    all_headlines = list_a + list_b
    tfidf_matrix = vectorizer.fit_transform(all_headlines)
    
    tfidf_a = tfidf_matrix[:len(list_a)]
    tfidf_b = tfidf_matrix[len(list_a):]
    similarity_matrix = cosine_similarity(tfidf_a, tfidf_b)
    
    a_covered = sum(1 for i in range(len(list_a)) if np.max(similarity_matrix[i]) >= threshold)
    b_covered = sum(1 for j in range(len(list_b)) if np.max(similarity_matrix[:, j]) >= threshold)
    
    return a_covered / len(list_a), b_covered / len(list_b)


def minhash_coverage(list_a, list_b, num_perm=128, threshold=0.6):
    """MinHashとLSHを使用したカバー率計算"""
    def get_shingles(text, k=2):
        text = re.sub(r'\s+', '', text.lower())
        return [text[i:i+k] for i in range(len(text)-k+1)]
    
    # MinHashオブジェクトを作成
    def create_minhash(text):
        m = MinHash(num_perm=num_perm)
        for shingle in get_shingles(text):
            m.update(shingle.encode('utf8'))
        return m
    
    # 各ヘッドラインのMinHashを計算
    minhashes_a = [create_minhash(text) for text in list_a]
    minhashes_b = [create_minhash(text) for text in list_b]
    
    # A→Bのカバー率を計算
    lsh_b = MinHashLSH(threshold=threshold, num_perm=num_perm)
    for i, mh in enumerate(minhashes_b):
        lsh_b.insert(i, mh)
    
    a_covered = sum(1 for mh_a in minhashes_a if lsh_b.query(mh_a))
    
    # B→Aのカバー率を計算
    lsh_a = MinHashLSH(threshold=threshold, num_perm=num_perm)
    for i, mh in enumerate(minhashes_a):
        lsh_a.insert(i, mh)
    
    b_covered = sum(1 for mh_b in minhashes_b if lsh_a.query(mh_b))
    
    return a_covered / len(list_a), b_covered / len(list_b)

def simhash_coverage(list_a, list_b, threshold=3):
    """SimHashを使用したカバー率計算"""
    def get_features(text):
        # 日本語テキストのn-gramを特徴量として使用
        text = text.lower()
        return [text[i:i+3] for i in range(len(text)-2)]
    
    # 各ヘッドラインのSimHashを計算
    hashes_a = [simhash.Simhash(get_features(text)) for text in list_a]
    hashes_b = [simhash.Simhash(get_features(text)) for text in list_b]
    
    # A→Bのカバー率を計算
    a_covered = sum(1 for h_a in hashes_a if 
                   any(h_a.distance(h_b) <= threshold for h_b in hashes_b))
    
    # B→Aのカバー率を計算
    b_covered = sum(1 for h_b in hashes_b if 
                   any(h_b.distance(h_a) <= threshold for h_a in hashes_a))
    
    return a_covered / len(list_a), b_covered / len(list_b)

def entity_coverage(list_a, list_b, threshold=0.3):
    """固有表現を利用したカバー率計算"""
    nlp = spacy.load("ja_core_news_sm")
    
    def get_entities(text):
        doc = nlp(text)
        return {(ent.text, ent.label_) for ent in doc.ents}
    
    # 一方向のカバー率を計算する関数
    def calc_directional_coverage(source_entities, target_entities):
        covered = 0
        for ent_src in source_entities:
            if not ent_src:
                continue
                
            if any(len(ent_src & ent_tgt) / len(ent_src | ent_tgt) >= threshold 
                  for ent_tgt in target_entities if ent_tgt):
                covered += 1
                
        return covered / len(source_entities) if source_entities else 0
    
    entities_a = [get_entities(text) for text in list_a]
    entities_b = [get_entities(text) for text in list_b]
    
    return (calc_directional_coverage(entities_a, entities_b), 
            calc_directional_coverage(entities_b, entities_a))

#######################################
# 統合関数
#######################################
def calculate_coverage(list_a, list_b, method='tfidf', **kwargs):
    """ヘッドラインカバー率を計算する統合関数"""
    methods = {
        'simple': simple_coverage,
        'ngram': ngram_coverage,
        'janome': janome_coverage,
        'tfidf': tfidf_coverage,
        'minhash': minhash_coverage,
        'simhash': simhash_coverage,
        'entity': entity_coverage
    }
    
    return methods[method](list_a, list_b, **kwargs)

#######################################
# メイン関数: 各手法を比較
#######################################
def compare_methods(list_a, list_b, show_details=True):
    """各カバー率計算手法の結果を比較して表示する"""
    if show_details:
        print("リストA:")
        for i, headline in enumerate(list_a):
            print(f"  A{i+1}: {headline}")
        print("\nリストB:")
        for i, headline in enumerate(list_b):
            print(f"  B{i+1}: {headline}")
        print("\n")
    
    print(f"リストサイズ: A={len(list_a)}件, B={len(list_b)}件")
    print("-" * 60)
    
    methods = [
        ('simple', {'threshold': 0.5}, 'シンプル文字列類似度'),
        ('ngram', {'n': 2, 'threshold': 0.3}, 'N-gram Jaccard係数'),
        ('janome', {'threshold': 0.4}, '形態素解析(Janome)'),
        ('tfidf', {'threshold': 0.5}, 'TF-IDF+余弦類似度'),
        ('minhash', {'threshold': 0.6}, 'MinHash/LSH'),
        ('simhash', {'threshold': 3}, 'SimHash'),
        ('entity', {'threshold': 0.3}, '固有表現抽出')
    ]
    
    print(f"{'手法':<20} {'A→Bのカバー率':<15} {'B→Aのカバー率':<15}")
    print("-" * 60)
    
    for method, kwargs, desc in methods:
        try:
            a_cov, b_cov = calculate_coverage(list_a, list_b, method=method, **kwargs)
            print(f"{desc:<20} {a_cov:.2%} {' '*8} {b_cov:.2%}")
        except Exception as e:
            print(f"{desc:<20} エラー: {str(e)}")
    
    print("-" * 60)

In [2]:
import numpy as np

def optimize_threshold_parameters(headlines_a1, headlines_b1, headlines_a2, headlines_b2, headlines_a4, headlines_b4):
    """
    各手法の閾値を最適化して、データセット1・2では高いカバー率、データセット4では低いカバー率になるようにする
    
    Args:
        headlines_a1, headlines_b1: データセット1（非常に類似した短いヘッドライン）
        headlines_a2, headlines_b2: データセット2（より長い非常に類似したヘッドライン）
        headlines_a4, headlines_b4: データセット4（全く似ていないヘッドライン）
        
    Returns:
        dict: 各手法の最適な閾値を含む辞書
    """
    # 手法と探索する閾値範囲の定義
    methods_and_ranges = {
        'simple': {'param': 'threshold', 'range': np.arange(0.1, 0.9, 0.05)},
        'ngram': {'param': 'threshold', 'range': np.arange(0.1, 0.9, 0.05), 'fixed': {'n': 2}},
        'janome': {'param': 'threshold', 'range': np.arange(0.1, 0.9, 0.05)},
        'tfidf': {'param': 'threshold', 'range': np.arange(0.1, 0.9, 0.05)},
        'minhash': {'param': 'threshold', 'range': np.arange(0.1, 0.9, 0.05), 'fixed': {'num_perm': 128}},
        'simhash': {'param': 'threshold', 'range': np.arange(1, 10, 1)},  # SimHashは整数値の範囲
        'entity': {'param': 'threshold', 'range': np.arange(0.1, 0.9, 0.05)}
    }
    
    # 最適な閾値を格納する辞書
    optimal_thresholds = {}
    
    print("閾値の最適化を開始...")
    
    # 各手法について最適な閾値を探索
    for method, settings in methods_and_ranges.items():
        best_score = -float('inf')
        best_threshold = None
        param_name = settings['param']
        
        print(f"{method}の最適な閾値を探索中...")
        
        for threshold in settings['range']:
            # この手法のパラメータを設定
            params = {param_name: threshold}
            if 'fixed' in settings:
                params.update(settings['fixed'])
            
            try:
                # 各データセットのカバー率を計算
                cov_a1_to_b1, cov_b1_to_a1 = calculate_coverage(headlines_a1, headlines_b1, method=method, **params)
                cov_a2_to_b2, cov_b2_to_a2 = calculate_coverage(headlines_a2, headlines_b2, method=method, **params)
                cov_a4_to_b4, cov_b4_to_a4 = calculate_coverage(headlines_a4, headlines_b4, method=method, **params)
                
                # 類似データセット（1と2）の平均カバー率
                similar_cov = (cov_a1_to_b1 + cov_b1_to_a1 + cov_a2_to_b2 + cov_b2_to_a2) / 4
                
                # 非類似データセット（4）の平均カバー率
                dissimilar_cov = (cov_a4_to_b4 + cov_b4_to_a4) / 2
                
                # スコア関数：類似データセットのカバー率を最大化し、非類似データセットとの差を最大化
                # 非類似データセットのカバー率には高いペナルティを与える
                score = similar_cov - 3 * dissimilar_cov
                
                if score > best_score:
                    best_score = score
                    best_threshold = threshold
            
            except Exception:
                continue
        
        # 最適な閾値を格納
        if best_threshold is not None:
            optimal_thresholds[method] = best_threshold
            print(f"  {method}: 最適な閾値 = {best_threshold}")
        else:
            print(f"  {method}: 最適な閾値が見つかりませんでした")
    
    return optimal_thresholds

# 最適な閾値でカバー率を計算して表示する関数
def show_coverage_with_optimal_thresholds(optimal_thresholds, headlines_a, headlines_b, dataset_name):
    """
    最適化された閾値を使用してカバー率を計算し表示する
    
    Args:
        optimal_thresholds: 最適化された閾値を含む辞書
        headlines_a, headlines_b: 比較するヘッドラインの2つのリスト
        dataset_name: データセットの説明
    """
    print(f"\n===== {dataset_name} (最適化閾値) =====")
    print(f"リストサイズ: A={len(headlines_a)}件, B={len(headlines_b)}件")
    print("-" * 70)
    
    methods_desc = {
        'simple': 'シンプル文字列類似度',
        'ngram': 'N-gram Jaccard係数',
        'janome': '形態素解析(Janome)',
        'tfidf': 'TF-IDF+余弦類似度',
        'minhash': 'MinHash/LSH',
        'simhash': 'SimHash',
        'entity': '固有表現抽出'
    }
    
    print(f"{'手法':<20} {'A→Bのカバー率':<15} {'B→Aのカバー率':<15} {'閾値':<10}")
    print("-" * 70)
    
    for method, threshold in optimal_thresholds.items():
        try:
            # 各手法に特有のパラメータを設定
            params = {'threshold': threshold}
            if method == 'ngram':
                params['n'] = 2
            elif method == 'minhash':
                params['num_perm'] = 128
            
            # カバー率を計算
            a_cov, b_cov = calculate_coverage(headlines_a, headlines_b, method=method, **params)
            print(f"{methods_desc[method]:<20} {a_cov:.2%} {' '*8} {b_cov:.2%} {' '*4} {threshold}")
        except Exception as e:
            print(f"{methods_desc[method]:<20} エラー: {str(e)}")
    
    print("-" * 70)

# メイン処理：閾値の最適化と各データセットでのカバー率比較
def optimize_and_compare_all_datasets():
    """
    すべてのデータセットに対して閾値を最適化し、結果を比較する
    """
    # データセット1: 非常に類似した短いヘッドライン
    headlines_a1 = [
        "日経平均が3.2%上昇、政府の経済対策を好感",
        "コロナ新変異株「XZ型」の感染例が国内で初確認、専門家が警戒呼びかけ",
        "東京オリンピック開催に向け組織委が最終準備、IOCが支持表明",
        "環境規制強化法案が可決、企業に対応求める",
        "米国FRBが利上げを発表、0.25%の利上げで市場予想通り",
        "GAFA4社の決算、クラウド事業が好調で増収増益に貢献",
        "都心の新築マンション価格が過去最高を更新、平均6500万円に",
        "女子サッカー代表が世界ランキング3位に浮上、監督が手腕を評価される",
        "大手銀行3行の統合決定、金融再編の動きが加速",
        "AI技術の特許出願が前年比50%増、競争激化を反映",
        "新薬の臨床試験結果が発表、成功率は当初予想を上回る",
        "プラスチック削減法が成立、使い捨て容器に新税導入へ",
        "テレワーク実施率が調査開始以来最高を記録、企業の67%が導入",
        "全国の賃金指数が前年同月比3.8%増、7年ぶりの高水準",
        "暗号資産市場が急回復、ビットコインは半年ぶり高値圏に",
        "新エネルギー関連株に資金流入、政府の脱炭素方針を好感",
        "観光客数が回復基調、インバウンド消費が地方経済を下支え",
        "工場の生産稼働率が過去最高を更新、部品供給問題が解消",
        "金融庁が投資家保護の新ガイドラインを発表、来年度から適用",
        "土地価格指数が全国平均で2.5%上昇、6年連続のプラス成長に"
    ]

    headlines_b1 = [
        "政府経済対策を受けて日経平均3.2%高、18カ月ぶり高値",
        "国内初、新型コロナ「XZ型」変異株の感染確認、重症化リスク調査中",
        "IOCが東京五輪開催を全面支持、準備は最終段階に",
        "新環境規制法案が可決、企業の排出削減義務化",
        "米連邦準備制度理事会が0.25%の利上げを決定、年内あと1回の見通し",
        "IT大手4社、クラウドサービスが牽引し第2四半期は予想上回る決算",
        "首都圏マンション平均価格が6500万円に到達、バブル期超え",
        "新電気自動車の予約開始、航続距離は競合の1.5倍に",
        "女子サッカーナショナルチーム、世界ランク3位に上昇、指揮官の采配が成功",
        "金融大手3行が経営統合を正式発表、業界再編が本格化",
        "人工知能関連特許の出願数が半数増、企業間の開発競争が激化",
        "新開発薬剤の治験成果を公表、有効性は予測を上回る結果に",
        "プラスチック規制新法が国会通過、使い捨て製品への課税が決定",
        "在宅勤務導入率67%で過去最高、調査開始以来の最高値を記録",
        "全国平均賃金が3.8%上昇、7年ぶりの大幅アップを記録",
        "仮想通貨相場が急騰、主要通貨ビットコインは6ヶ月ぶりの高値に",
        "脱炭素関連企業の株価が上昇、政府方針を受けて投資家の関心高まる",
        "訪日客数の回復が鮮明に、外国人観光消費が地域経済を活性化",
        "製造業の工場稼働率が記録的水準に、部品調達が正常化",
        "金融当局が投資家向け新保護指針を策定、翌年度より実施",
        "地価全国平均2.5%上昇、6年連続で前年を上回る"
    ]

    # データセット2: より長い非常に類似したヘッドライン
    headlines_a2 = [
        "政府は総額20兆円規模の新たな経済対策を閣議決定、企業の設備投資減税や個人消費喚起策を含む",
        "世界保健機関(WHO)は新型コロナウイルスの新変異株「XZ型」について専門家会議を開催、現時点で深刻な懸念はないと発表",
        "東京五輪の開催1か月前、IOCと組織委員会が安全対策を強化、無観客開催の可能性も検討",
        "大手自動車メーカーA社が新型電気自動車を発表、航続距離800kmで業界最長、価格は450万円から",
        "台風19号が週末に西日本に接近の見込み、気象庁は高波と大雨に警戒するよう呼びかけ",
        "中央銀行が金融政策決定会合で現行の金融緩和策維持を決定、市場は好感して株高に",
        "国内の主要携帯電話会社3社が新料金プランを発表、データ通信量無制限で月額料金は平均15%引き下げへ、総務省の指導に対応",
        "次世代半導体の製造技術で日本企業連合が共同開発プロジェクトを始動、政府が5年間で総額1兆円の支援を表明、国際競争力回復目指す",
        "医療保険制度改革法案が可決、高齢者の自己負担割合を段階的に引き上げ、2025年度から完全施行、財政健全化が目的",
        "人工知能を活用した新型の気象予測システムを気象庁が導入、予測精度が従来比30%向上、局地的豪雨の早期警戒に期待",
        "大手商社5社の4-6月期決算が出揃い、全社が最高益を更新、資源価格高騰と円安が追い風、株主還元も積極化",
        "国立大学の授業料を世帯年収に応じて無償化する新制度が2024年度からスタート、年収700万円未満が対象に",
        "再生可能エネルギーの発電比率が初めて30%を突破、太陽光と風力の設備増強が寄与、2030年目標の40%達成に前進",
        "国内最大の電子商取引企業が物流センターの完全自動化を発表、AI制御のロボットが商品の仕分け・梱包を担当、人手不足解消へ",
        "世界最速となる次世代通信規格「6G」の国際標準化で日米欧連合が合意、2030年の実用化目指し共同研究開発へ",
        "大都市圏の鉄道各社が運賃改定を申請、10月から平均12%値上げへ、人件費高騰とエネルギーコスト増が理由",
        "海洋プラスチック削減に向けた国際条約が発効、日本も批准、2035年までに海洋流出量を80%削減する目標",
        "新型宇宙望遠鏡の観測データから太陽系外の地球型惑星に水の存在を示す証拠を発見、生命存在の可能性に期待高まる",
        "政府が子育て支援の新パッケージを発表、第2子以降の保育料完全無償化や児童手当増額など、少子化対策を強化"
    ]

    headlines_b2 = [
        "閣議決定された20兆円規模の経済対策、企業投資減税と消費喚起策の二本柱で景気回復目指す",
        "WHO専門家会議、コロナ新変異株「XZ型」は感染力がやや強いが現時点で深刻な脅威ではないと結論",
        "東京オリンピック開幕まで1ヶ月、感染対策強化でIOCと組織委が合意、無観客選択肢も",
        "A社が次世代EV「エコフューチャー」を正式発表、一充電あたり800kmの走行を実現、税込450万円から",
        "気象庁：台風19号が週末に九州・四国地方に接近、暴風雨に警戒を",
        "金融政策決定会合、ゼロ金利政策と資産買入れ継続を決定、インフレ目標には届かず",
        "ノーベル物理学賞に日米の共同研究チーム、量子コンピューティングの研究で受賞",
        "大手携帯3社、新たな料金体系を一斉発表、データ使い放題で平均15%値下げ、総務省からの要請受け改定",
        "国内企業連合による次世代半導体開発プロジェクト発足、政府が1兆円規模の資金援助を決定、技術覇権競争に参戦",
        "医療保険改革法が成立、高齢者負担率を順次引き上げ、25年度より全面適用、財政再建へ一歩前進",
        "気象庁がAI搭載の新予報システム運用開始、従来より予測精度30%向上、ゲリラ豪雨対策に効果期待",
        "主要商社5社、第1四半期で過去最高益を達成、資源高と円安効果で収益拡大、配当増額も発表",
        "国立大の学費無償化制度が確定、年収700万円以下の世帯対象、24年度入学生から適用へ",
        "再エネ発電シェアが初の30%突破、太陽光・風力発電所の増設が進み、30年目標の40%に向け順調",
        "EC大手が物流施設の無人化技術を導入、AIロボットによる完全自動化で出荷処理、人員不足を解消",
        "6G通信規格の共同開発に日米欧が基本合意、次世代通信技術で連携、30年商用化を目標に研究加速",
        "首都圏の鉄道会社が運賃引き上げを正式申請、10月より約12%値上げ実施へ、コスト増加が主因",
        "海洋プラスチック対策国際条約が正式発効、日本も参加表明、35年までに流出量8割減を目指す",
        "新観測機器が系外惑星の大気中に水分子を検出、生命存在の可能性を示す重要な発見と専門家",
        "子育て支援強化策を政府が正式発表、第2子以降の保育完全無料化など、少子化に歯止めをかける狙い"
    ]

    # データセット4: 全く似ていないヘッドライン
    headlines_a4 = [
        "国内最大規模の美術展が東京で開幕、欧州の名画80点を展示",
        "高速道路の渋滞予測システムがAI活用で精度向上、連休前に実用化",
        "新種の深海生物を発見、南太平洋の調査で10種以上の未知生物を確認",
        "プロ野球選手の年俸調査、平均年収は4500万円で前年比3%増",
        "世界的ピアニストが40年ぶりに来日公演、チケットは発売1分で完売",
        "特定外来生物の駆除作戦が始動、地域住民とNPOが協力",
        "月面探査機が新たな氷の痕跡を発見、将来の月面基地建設に期待",
        "古代遺跡から新たな文字が刻まれた石板を発掘、解読作業が進行中",
        "食物アレルギー治療の新薬が治験で高い効果、来年にも実用化へ",
        "スポーツ栄養学の国際学会が初の日本開催、最新研究成果を発表",
        "希少な蝶の新生息地を確認、保全活動が実を結ぶ",
        "伝統音楽の保存プロジェクトが発足、無形文化財の継承に取り組む",
        "火山活動の予測精度が向上、新システム導入で前兆現象を早期検知",
        "絶滅危惧種の保護区域を拡大、生息数の回復傾向が報告される",
        "古典文学の復刻版が異例のベストセラーに、若年層の関心高まる",
        "宇宙線観測施設が完成、宇宙の謎解明に期待",
        "南極の氷床調査で新データ、温暖化の影響を詳細に把握",
        "新作映画が興行収入記録を更新、シリーズ最高のヒット作に",
        "独自の農法で収穫量50%増、農業革新として注目される",
        "珍しい天体ショーが今夜観測可能、200年ぶりの現象と専門家",
        "考古学者が失われた古代都市の遺構を発見、新たな歴史的知見が得られる"
    ]

    headlines_b4 = [
        "農業支援ドローンの販売が急増、人手不足の農家に導入広がる",
        "児童向け体験型科学館がオープン、最新技術で宇宙や海洋を疑似体験",
        "電子書籍市場が前年比30%増、紙の出版物を初めて上回る",
        "伝統工芸の担い手育成プログラム開始、若手職人の減少に歯止め",
        "国内最大のファッションイベント開催、サステナブルデザインに注目集まる",
        "オンライン診療の利用者が1年で倍増、地方の医師不足解消に貢献",
        "史上最大の恐竜化石を発掘、体長30メートル以上と推定される",
        "人工衛星による環境モニタリング技術が進化、森林減少を精密に追跡",
        "休眠火山の地下マグマ活動に変化、専門家チームが監視強化",
        "仮想現実技術を用いた教育プログラムが全国展開、没入型学習に成果",
        "新たな遺伝子療法が臨床試験で有望な結果、難病治療に光",
        "海洋深層水を活用した新産業が地方創生の柱に、雇用創出効果も",
        "長寿研究の国際会議が招致決定、世界の専門家が一堂に会する",
        "気候変動による生態系への影響調査結果を公表、保全策の見直しへ",
        "最新のスーパーコンピュータが稼働開始、創薬研究の加速に期待",
        "次世代バッテリー技術の特許を公開、充電時間を従来の10分の1に短縮",
        "熱帯雨林の未踏地域で新種の哺乳類を発見、学術的価値が高いと評価",
        "古代DNA分析から新事実判明、縄文人と弥生人の関係に新説",
        "ワクチン新技術の開発に成功、複数の感染症に対応可能に",
        "世界文化遺産の修復プロジェクトが完了、10年の歳月をかけた保存作業",
        "最新の海底探査で沈没船を発見、江戸時代の交易船と特定"
    ]
    
    # 閾値の最適化
    optimal_thresholds = optimize_threshold_parameters(
        headlines_a1, headlines_b1,
        headlines_a2, headlines_b2,
        headlines_a4, headlines_b4
    )
    
    # 最適な閾値の一覧表示
    print("\n===== 最適化された閾値 =====")
    methods_desc = {
        'simple': 'シンプル文字列類似度',
        'ngram': 'N-gram Jaccard係数',
        'janome': '形態素解析(Janome)',
        'tfidf': 'TF-IDF+余弦類似度',
        'minhash': 'MinHash/LSH',
        'simhash': 'SimHash',
        'entity': '固有表現抽出'
    }
    
    for method, threshold in optimal_thresholds.items():
        print(f"{methods_desc[method]}: {threshold}")
    
    # 各データセットでの最適化閾値によるカバー率の比較
    show_coverage_with_optimal_thresholds(
        optimal_thresholds, headlines_a1, headlines_b1, 
        "データセット1: 非常に類似した短いヘッドライン"
    )
    show_coverage_with_optimal_thresholds(
        optimal_thresholds, headlines_a2, headlines_b2, 
        "データセット2: より長い非常に類似したヘッドライン"
    )
    show_coverage_with_optimal_thresholds(
        optimal_thresholds, headlines_a4, headlines_b4, 
        "データセット4: 全く似ていないヘッドライン"
    )
    
    return optimal_thresholds

if __name__ == "__main__":
    # 閾値の最適化とカバー率比較を実行
    optimize_and_compare_all_datasets()

閾値の最適化を開始...
simpleの最適な閾値を探索中...
  simple: 最適な閾値 = 0.3500000000000001
ngramの最適な閾値を探索中...
  ngram: 最適な閾値 = 0.1
janomeの最適な閾値を探索中...
  janome: 最適な閾値 = 0.20000000000000004
tfidfの最適な閾値を探索中...
  tfidf: 最適な閾値 = 0.1
minhashの最適な閾値を探索中...
  minhash: 最適な閾値 = 0.20000000000000004
simhashの最適な閾値を探索中...
  simhash: 最適な閾値 = 1
entityの最適な閾値を探索中...
  entity: 最適な閾値 = 0.1

===== 最適化された閾値 =====
シンプル文字列類似度: 0.3500000000000001
N-gram Jaccard係数: 0.1
形態素解析(Janome): 0.20000000000000004
TF-IDF+余弦類似度: 0.1
MinHash/LSH: 0.20000000000000004
SimHash: 1
固有表現抽出: 0.1

===== データセット1: 非常に類似した短いヘッドライン (最適化閾値) =====
リストサイズ: A=20件, B=21件
----------------------------------------------------------------------
手法                   A→Bのカバー率        B→Aのカバー率        閾値        
----------------------------------------------------------------------
シンプル文字列類似度           80.00%          71.43%      0.3500000000000001
N-gram Jaccard係数     100.00%          95.24%      0.1
形態素解析(Janome)        90.00%          85.71%      0.20000000000000004


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def find_duplicate_news(headlines, threshold=0.1):
    """TF-IDFベースでニュースの重複を検出する"""
    if not headlines:
        return []
    
    # TF-IDFと類似度計算
    vectorizer = TfidfVectorizer(analyzer='char', ngram_range=(2, 3))
    tfidf_matrix = vectorizer.fit_transform(headlines)
    sim_matrix = cosine_similarity(tfidf_matrix)
    np.fill_diagonal(sim_matrix, 0)  # 自分自身との比較を除外
    
    # 類似グループの検出（単純なアルゴリズム）
    n = len(headlines)
    visited = [False] * n
    groups = []
    
    for i in range(n):
        if visited[i]:
            continue
            
        # 類似度が閾値以上のインデックスを見つける
        group = [i]
        visited[i] = True
        
        for j in range(n):
            if not visited[j] and sim_matrix[i, j] >= threshold:
                group.append(j)
                visited[j] = True
        
        # サイズが2以上のグループのみ保存
        if len(group) > 1:
            groups.append(sorted(group))
    
    return groups

def print_duplicate_groups(headlines, groups):
    """検出された重複グループを表示"""
    if not groups:
        print("重複するニュースは検出されませんでした。")
        return
    
    print(f"検出された類似グループ数: {len(groups)}")
    
    for i, group in enumerate(sorted(groups, key=len, reverse=True)):
        print(f"\nグループ {i+1} ({len(group)}件):")
        for idx in group:
            print(f"  [{idx}] {headlines[idx]}")

# 使用例
if __name__ == "__main__":
    # サンプルヘッドライン
    headlines = [
        # 日経平均関連（類似グループ1）
        "日経平均が3.2%上昇、政府の経済対策を好感",
        "政府経済対策を受けて日経平均3.2%高、18カ月ぶり高値",
        "経済対策発表後、東証で日経平均株価が急伸、3%超の上昇",
        
        # コロナ変異株関連（類似グループ2）
        "コロナ新変異株「XZ型」の感染例が国内で初確認",
        "国内初、新型コロナ「XZ型」変異株の感染確認、重症化リスク調査中",
        "XZ型変異株、国内で感染拡大の兆し、専門家会議が対応検討",
        
        # 自動車業界関連（類似グループ3）
        "自動車大手A社が電気自動車の新モデルを発表、航続距離は業界最長",
        "A社が次世代EV発表、一回の充電で700km走行可能と発表",
        "大手自動車メーカーの新型EVが話題、競合他社を上回る性能で市場に挑戦",
        
        # 金融政策関連（類似グループ4）
        "中央銀行が政策金利を据え置き、経済動向を注視する姿勢",
        "日銀、金融政策の現状維持を決定、市場予想通りの判断",
        
        # 災害・防災関連（類似グループ5）
        "東京都が新たな防災計画を発表、AI活用で避難誘導を効率化",
        "首都直下型地震に備え、東京都が最新AI技術導入の防災計画を策定",
        
        # 以下は類似性のない単独ニュース
        "国内最大規模の美術展が東京で開幕、欧州の名画80点を展示",
        "プロ野球選手の年俸調査、平均年収は4500万円で前年比3%増",
        "世界的ピアニストが40年ぶりに来日公演、チケットは発売1分で完売",
        "南極の氷床調査で新データ、温暖化の影響を詳細に把握",
        "新作映画が興行収入記録を更新、シリーズ最高のヒット作に",
        "ノーベル物理学賞に日米の共同研究チーム、量子コンピューティングの研究で受賞"
    ]
    
    # 重複検出と表示
    groups = find_duplicate_news(headlines)
    print_duplicate_groups(headlines, groups)

検出された類似グループ数: 4

グループ 1 (3件):
  [0] 日経平均が3.2%上昇、政府の経済対策を好感
  [1] 政府経済対策を受けて日経平均3.2%高、18カ月ぶり高値
  [2] 経済対策発表後、東証で日経平均株価が急伸、3%超の上昇

グループ 2 (3件):
  [3] コロナ新変異株「XZ型」の感染例が国内で初確認
  [4] 国内初、新型コロナ「XZ型」変異株の感染確認、重症化リスク調査中
  [5] XZ型変異株、国内で感染拡大の兆し、専門家会議が対応検討

グループ 3 (2件):
  [6] 自動車大手A社が電気自動車の新モデルを発表、航続距離は業界最長
  [8] 大手自動車メーカーの新型EVが話題、競合他社を上回る性能で市場に挑戦

グループ 4 (2件):
  [11] 東京都が新たな防災計画を発表、AI活用で避難誘導を効率化
  [12] 首都直下型地震に備え、東京都が最新AI技術導入の防災計画を策定
